In [1]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from src.pcd_de_noising import MistNet
from src.pcd_de_noising import PCDDataset, PointCloudDataModule
from torch.utils.data import DataLoader
import os
import torch





# Configuration

In [2]:
NUMBER_GPUS = 1
DATASET_PATH = "./rayz"

# Logging

In [3]:
# Create Tensorboard logger
logger = TensorBoardLogger("./log", name="MistNet")

# Model

### Create new model

In [4]:
model = MistNet()

### Load from checkpoint

In [4]:
model = MistNet.load_from_checkpoint("log/MistNet/version_2/checkpoints/epoch=49-step=5650.ckpt")

# Prepare Data

In [5]:
data_module = PointCloudDataModule(
    os.path.join(DATASET_PATH, "train"),
    os.path.join(DATASET_PATH, "val"),
    os.path.join(DATASET_PATH, "test")
)

# Overfit

In [18]:
trainer = pl.Trainer(
    logger=logger,
    overfit_batches=51,
    max_epochs=50,
    log_every_n_steps=1
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model, data_module)


  | Name              | Type                       | Params
-----------------------------------------------------------------
0 | lila1             | LiLaBlock                  | 55.1 K
1 | lila2             | LiLaBlock                  | 804 K 
2 | lila3             | LiLaBlock                  | 2.2 M 
3 | lila4             | LiLaBlock                  | 4.2 M 
4 | dropout           | Dropout2d                  | 0     
5 | lila5             | LiLaBlock                  | 2.0 M 
6 | classifier        | Conv2d                     | 387   
7 | accuracy          | MulticlassAccuracy         | 0     
8 | average_precision | MulticlassAveragePrecision | 0     
-----------------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.281    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Val found 51 files


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Train found 51 files                                                       


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:268: You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.
/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 49: 100%|██████████| 13/13 [00:55<00:00,  0.23it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:55<00:00,  0.23it/s, v_num=0]


# Train

In [6]:
trainer = pl.Trainer(
    logger=logger,
    max_epochs=50,
    log_every_n_steps=1,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model, data_module)


  | Name              | Type                       | Params
-----------------------------------------------------------------
0 | lila1             | LiLaBlock                  | 72.4 K
1 | lila2             | LiLaBlock                  | 804 K 
2 | lila3             | LiLaBlock                  | 2.2 M 
3 | lila4             | LiLaBlock                  | 4.2 M 
4 | dropout           | Dropout2d                  | 0     
5 | lila5             | LiLaBlock                  | 2.0 M 
6 | classifier        | Conv2d                     | 1.0 K 
7 | accuracy          | MulticlassAccuracy         | 0     
8 | average_precision | MulticlassAveragePrecision | 0     
-----------------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.352    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Val found 450 files


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Train found 450 files                                                      


/Users/xavier/micromamba/envs/pcd-de-noising/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 49: 100%|██████████| 113/113 [16:19<00:00,  0.12it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 113/113 [16:19<00:00,  0.12it/s, v_num=2]


# Test

In [1]:
trainer.test(model, datamodule=data_module)

NameError: name 'trainer' is not defined

# Measure Inference Time

In [ ]:
model.freeze()
model = model.cuda()

In [ ]:
loader = data_module.test_dataloader()
start, end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
reps = 100
timings = [0] * reps

distance, reflectivity, labels = next(iter(loader))
distance = distance.cuda()
reflectivity = reflectivity.cuda()
labels = labels.cuda()

for rep in range(reps):
    start.record()
    _ = model(distance, reflectivity)
    end.record()

    # Wait for GPU sync
    torch.cuda.synchronize()
    curr_time = start.elapsed_time(end)
    timings[rep] = curr_time / 1000  # ms to s

images_processed = reps * loader.batch_size
mean_time = sum(timings) / images_processed
f"Mean inference time: {mean_time:.2f}s, mean FPS: {1 / mean_time:.2f}"